In [ ]:
from river_torch.anomaly import Autoencoder
from river import datasets, metrics
from river_torch.anomaly.nn_builder import get_fc_encoder, get_fc_decoder

In [1]:
encoder_fn = get_fc_encoder
decoder_fn = get_fc_decoder
dataset = datasets.CreditCard().take(5000)
metric = metrics.ROCAUC()

model = Autoencoder(encoder_fn=encoder_fn,decoder_fn=decoder_fn, lr=0.01)

for x,y in dataset:
    score = model.score_one(x)
    metric.update(y_true=y, y_pred=score)
    model.learn_one(x=x)
print(f'ROCAUC: {metric.get()}')

NameError: name 'metrics' is not defined